In [97]:
# unzip input file first

import xml.etree.ElementTree as ET
import requests
import math
import pprint
import time
import numpy as np


### Mapping

In [3]:
root = ET.parse('input-files/RoughlyTravisCounty.osm').getroot()

In [4]:
latitudes = []
longitudes = []
for type_tag in root.findall("node"):
    d = type_tag.attrib
    latitudes.append(float(d["lat"]))
    longitudes.append(float(d["lon"]))

In [5]:
LATITUDE_MILES = 69
LONGITUDE_MILES = 54.6

vertical_distance = (max(latitudes) - min(latitudes))*LATITUDE_MILES
horizontal_distance = (max(longitudes) - min(longitudes))*LONGITUDE_MILES

In [6]:
# Assuming we want to make 400 squares, how should we partition the vertically/horizontally
NUM_REGIONS = 400
square_len = (vertical_distance*horizontal_distance/NUM_REGIONS)**0.5

In [7]:
latitude_delta = square_len/LATITUDE_MILES
longitude_delta = square_len/LONGITUDE_MILES
longitude_delta

0.039167692270157484

In [8]:
lat_lon = zip(latitudes, longitudes)

In [9]:
min_lat = min(latitudes)
min_lon = min(longitudes)
max_lat = max(latitudes)
max_lon = max(longitudes)
nrows = math.ceil((max_lat - min_lat) / latitude_delta)
ncols = math.ceil((max_lon - min_lon) / longitude_delta)
def map_point_to_region(latitude, longitude):
    return math.floor((latitude-min_lat)/latitude_delta) * ncols  + math.floor((longitude-min_lon)/longitude_delta)

In [41]:
def get_representative(region_num): # 8 cols per row, # R10 --> row_num = 1, col_num = 2
    row_num = region_num//ncols # Correct
    col_num = region_num - row_num*ncols
    lat = min_lat + row_num * latitude_delta + 0.5*latitude_delta
    lon = min_lon + col_num * longitude_delta + 0.5*longitude_delta
    return [lon, lat]

In [11]:
map_point_to_region(30.084446, -97.702885)

13

In [12]:
get_representative(13)

[30.090783782593846, -97.68468325435288]

In [40]:
for i in range(400):
    get_representative(i)

### Open Route Service API

In [46]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624884518d9d4075408e95e3401165976247',
    'Content-Type': 'application/json; charset=utf-8'
}

In [80]:
# Distance matrix can be many to many, one to many, or many to one
# Can specify index of sources and destinations

sources = range(0,8)

#body = {"locations": [get_representative(i) for i in range(400)], "sources":[0]}
#body = {"locations": [get_representative(i) for i in range(400)], "sources":[0,1]}
body = {"locations": [get_representative(i) for i in range(400)], "sources":list(range(0,8))}

range(0, 8)


In [81]:
# Limit of 3500 routes per api call. So max of 8 columns at a time
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

In [82]:
# 400 x 400 = 160,000
#pprint.pprint(call._content)
#pprint.pprint(call.json())

In [86]:
result = call.json()['durations']
pprint.pprint(result)


9


### Batch calls of distance matrix

In [95]:
locations = [get_representative(i) for i in range(400)]

durations = []

for i in range(50):
    start = i * 8
    sources = list(range(start, start + 8))
    
    body = {"locations": locations, "sources": sources}
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
    
    duration_batch = call.json()['durations']
    for j in range(len(duration_batch)):
        durations.append(duration_batch[j])
        
    time.sleep(3)
    
pprint.pprint(durations)


[[0.0,
  1626.44,
  2589.26,
  2807.65,
  3035.54,
  3058.36,
  4173.38,
  3709.69,
  3636.23,
  3682.96,
  3858.82,
  4180.79,
  4241.53,
  4379.98,
  4795.82,
  4933.35,
  5580.82,
  5103.28,
  5233.52,
  5501.2,
  5933.48,
  5889.49,
  1760.52,
  2500.23,
  2223.21,
  2147.09,
  3030.45,
  3128.5,
  3314.98,
  5005.05,
  4009.15,
  4446.13,
  3846.2,
  4546.2,
  4584.22,
  4455.29,
  4560.09,
  5173.85,
  5285.31,
  5353.79,
  5418.9,
  5706.35,
  5691.38,
  6036.73,
  3077.06,
  2734.64,
  2884.71,
  2602.87,
  2709.68,
  2976.2,
  3857.94,
  4141.41,
  4416.84,
  4217.67,
  4235.29,
  4269.67,
  4670.25,
  4684.09,
  4700.45,
  5118.02,
  5245.24,
  7246.88,
  5264.24,
  5739.49,
  5650.56,
  7200.42,
  2462.0,
  2716.9,
  2453.52,
  2669.84,
  3236.95,
  3153.45,
  3675.84,
  3821.97,
  3873.22,
  4107.72,
  4314.67,
  4380.4,
  4903.0,
  4765.06,
  4752.04,
  4827.04,
  4929.71,
  5285.65,
  5968.37,
  6501.07,
  6237.06,
  6640.93,
  3181.58,
  2601.5,
  2761.39,
  2711.24,
  3

  3701.05,
  3818.88,
  4131.46,
  6035.14,
  6170.45,
  5889.32,
  4542.24,
  4335.28,
  4432.95,
  4328.05,
  3777.67,
  3631.0,
  3917.92,
  4030.81,
  3466.23,
  3225.74,
  3019.85,
  3195.11,
  3111.68,
  3767.8,
  3659.55,
  3852.19,
  3983.57,
  4090.09,
  4480.05,
  None,
  6084.8,
  5409.59,
  4468.47],
 [3682.96,
  4389.82,
  3243.77,
  3873.1,
  4100.99,
  2151.27,
  2020.5,
  854.18,
  626.85,
  0.0,
  492.62,
  814.59,
  933.96,
  1072.4,
  1488.24,
  1625.78,
  2273.25,
  1957.16,
  2087.39,
  2355.08,
  2787.36,
  2743.36,
  3931.73,
  3092.33,
  2591.61,
  2515.5,
  1729.89,
  1351.65,
  1055.31,
  1955.17,
  945.94,
  1079.93,
  575.32,
  1215.79,
  1276.64,
  1147.71,
  1252.52,
  1866.27,
  1977.74,
  2062.22,
  2272.78,
  2523.17,
  2526.95,
  2890.61,
  4119.05,
  3315.29,
  2744.95,
  2373.89,
  1884.69,
  1691.74,
  1932.13,
  1485.69,
  1373.79,
  1082.84,
  956.59,
  998.8,
  1399.37,
  1376.51,
  1392.87,
  1810.44,
  1937.67,
  3963.99,
  1981.35,
  2456.6,
 

  2668.3,
  2526.35,
  3053.49,
  2968.83,
  3383.48,
  3836.16,
  3610.22,
  5417.26,
  4932.17,
  4973.01,
  5396.86,
  7229.07,
  5144.96,
  4620.67,
  5284.07,
  4018.0,
  3551.95,
  3186.33,
  3256.86,
  3143.48,
  3079.38,
  2794.18,
  2558.22,
  2666.51,
  3065.12,
  3581.53,
  3410.84,
  3527.02,
  3396.29,
  5175.31,
  5001.28,
  6707.66,
  6851.31,
  6980.03,
  4819.19,
  5295.27,
  4266.9,
  4534.85,
  3472.72,
  3148.63,
  3381.16,
  2990.28,
  3025.24,
  2897.44,
  2798.86,
  2944.36,
  3216.65,
  3424.83,
  3453.36,
  3697.42,
  4048.83,
  5881.48,
  6977.4,
  6000.4,
  5464.77,
  5313.96,
  5232.81,
  5020.38,
  4422.89,
  3992.07,
  3570.64,
  3540.25,
  3434.38,
  3289.56,
  3079.98,
  3121.64,
  2985.12,
  2745.03,
  3138.07,
  3352.41,
  3550.76,
  3826.03,
  4033.87,
  5966.35,
  6116.74,
  5864.4,
  6165.11,
  5715.18,
  None,
  4857.37,
  4326.95,
  3941.15,
  3674.63,
  3729.75,
  3809.02,
  3254.77,
  3202.33,
  3291.73,
  3031.75,
  2849.05,
  3022.21,
  3280.9

  4010.42,
  4221.3,
  4235.25,
  3276.52,
  3395.33,
  3416.19,
  4747.79,
  5190.08,
  3043.96,
  3397.07,
  3150.91,
  2973.27,
  2568.78,
  2675.57,
  2632.64,
  3205.25,
  3445.9,
  3463.1,
  3706.42,
  3791.48,
  4206.13,
  4658.81,
  4749.39,
  4179.37,
  3694.29,
  3735.13,
  4158.98,
  6862.27,
  4778.16,
  4253.87,
  4917.27,
  3651.21,
  3185.16,
  2822.61,
  3098.88,
  2905.45,
  2783.16,
  2996.19,
  3271.98,
  3253.59,
  3697.0,
  4404.18,
  4233.48,
  4666.19,
  4535.45,
  3937.43,
  3763.39,
  6340.86,
  6484.52,
  6613.23,
  4452.39,
  4928.48,
  3900.1,
  4168.06,
  3105.92,
  2781.83,
  3247.76,
  2827.36,
  3004.33,
  3058.4,
  3128.35,
  3570.09,
  3780.8,
  4050.56,
  4276.01,
  4631.67,
  5078.71,
  4643.6,
  6010.91,
  5633.6,
  5097.97,
  4947.16,
  4866.02,
  4653.59,
  4056.09,
  3625.27,
  3203.84,
  3173.45,
  3300.98,
  3126.64,
  3110.5,
  3451.33,
  3314.61,
  3370.76,
  3763.8,
  3978.14,
  4270.73,
  4451.76,
  4775.8,
  4501.0,
  5150.25,
  5442.13,
 

  5123.5,
  3496.55,
  4147.07,
  2982.64,
  2762.79,
  2480.91,
  2343.56,
  2426.52,
  2275.18,
  1967.3,
  1855.84,
  2125.51,
  2756.35,
  2513.83,
  2922.17,
  2677.78,
  4949.2,
  4963.15,
  4004.42,
  4123.23,
  4144.08,
  4604.6,
  5046.89,
  3771.85,
  3393.19,
  3040.14,
  2830.08,
  2425.59,
  2653.95,
  2540.18,
  2492.47,
  2158.32,
  2016.37,
  2543.5,
  2458.85,
  2873.49,
  3326.17,
  3416.75,
  4907.27,
  4422.19,
  4463.02,
  4886.88,
  6719.09,
  4634.97,
  4110.68,
  4774.08,
  3508.02,
  3041.97,
  2676.35,
  2746.88,
  2633.49,
  2569.39,
  2284.2,
  2048.23,
  2156.53,
  2555.13,
  3071.54,
  2900.85,
  3333.56,
  3202.82,
  4665.33,
  4491.29,
  6197.67,
  6341.33,
  6470.04,
  4309.2,
  4785.29,
  3756.92,
  4024.87,
  2962.73,
  2638.64,
  2871.18,
  2480.29,
  2515.25,
  2387.45,
  2288.88,
  2434.38,
  2706.67,
  2914.84,
  2943.38,
  3359.01,
  3810.92,
  5371.49,
  6467.42,
  5490.41,
  4954.79,
  4803.98,
  4722.83,
  4510.4,
  3912.9,
  3482.08,
  3060.6

...

  2619.93,
  2815.45,
  2807.85,
  2759.03,
  6002.11,
  5955.33,
  5725.28,
  4630.07,
  4071.78,
  3494.68,
  3216.67,
  2961.47,
  2915.03,
  2613.26,
  2656.02,
  1941.68,
  1881.49,
  1846.19,
  1673.43,
  1479.94,
  1665.33,
  1933.45,
  2434.45,
  2428.29,
  2667.38,
  2649.22,
  6019.23,
  5021.86,
  4662.15,
  4333.3,
  3646.21,
  3455.19,
  3295.57,
  2981.72,
  3666.71,
  2785.69,
  2285.78,
  1936.38,
  1960.06,
  1777.91,
  1609.0,
  1294.89,
  1482.09,
  1649.15,
  1843.49,
  2272.63,
  2191.26,
  2229.04,
  5341.94,
  5639.61,
  5254.24,
  3926.5,
  3945.46,
  3495.52,
  3273.31,
  3269.48,
  2762.68,
  2875.89,
  2113.71,
  1858.96,
  1775.09,
  1782.86,
  1838.76,
  1371.52,
  1464.89,
  1760.72,
  2047.6,
  2249.1,
  2261.88,
  2297.03,
  5603.21,
  5340.41,
  5304.15,
  4617.96,
  4278.67,
  4820.22,
  3836.48,
  2756.78,
  3344.53,
  2319.09,
  2219.88,
  1914.21,
  1641.64,
  1729.86,
  1596.36,
  1207.05,
  1144.58,
  1751.63,
  2254.03,
  2326.08,
  2650.32,
  22

 [6803.56,
  7258.23,
  5677.07,
  6370.42,
  6598.31,
  4917.7,
  5228.03,
  4061.71,
  3834.38,
  3779.73,
  3523.02,
  3356.8,
  3101.15,
  2835.43,
  3088.86,
  3195.02,
  3705.5,
  3736.74,
  3732.14,
  3729.77,
  3981.09,
  3824.86,
  6365.02,
  5525.62,
  5024.9,
  4948.79,
  4480.0,
  4290.73,
  4262.84,
  4980.9,
  3971.67,
  3909.93,
  3416.54,
  3571.67,
  3443.84,
  2900.19,
  2847.68,
  3196.76,
  3403.13,
  3248.76,
  3375.37,
  3552.11,
  3555.89,
  3972.11,
  6218.24,
  5562.93,
  5178.25,
  4807.18,
  4309.98,
  4091.04,
  4223.31,
  3776.88,
  3945.31,
  3742.59,
  3402.7,
  3399.2,
  3226.41,
  2798.62,
  2664.1,
  3081.67,
  3124.2,
  4992.92,
  3010.29,
  3485.54,
  3515.06,
  4899.2,
  5144.05,
  5029.14,
  4545.95,
  4564.66,
  4630.77,
  4168.13,
  4466.2,
  3665.98,
  3215.79,
  3475.31,
  3364.8,
  3273.76,
  3293.97,
  2816.52,
  2579.88,
  2573.09,
  2675.76,
  3031.7,
  3714.42,
  3758.38,
  4036.74,
  3741.71,
  5464.52,
  4689.79,
  4661.91,
  4511.48,
  

  2280.98,
  2216.89,
  2019.86,
  1822.13,
  1727.5,
  2124.72,
  1945.34,
  1767.9,
  1452.72,
  1363.38,
  5959.6,
  5785.56,
  5916.87,
  6060.53,
  6189.24,
  4028.4,
  4504.49,
  3476.12,
  3744.07,
  2671.71,
  2362.95,
  2518.67,
  2127.79,
  2117.45,
  1835.19,
  1653.13,
  1665.85,
  1725.23,
  1334.78,
  1425.84,
  1110.76,
  1557.8,
  6665.77,
  5693.73,
  5209.61,
  4673.99,
  4523.18,
  4442.03,
  4229.6,
  3632.11,
  3201.28,
  2779.86,
  2662.6,
  2571.88,
  2295.12,
  1875.01,
  1625.57,
  1530.37,
  1206.9,
  1293.02,
  1035.39,
  905.99,
  972.82,
  1254.89,
  5950.7,
  4833.08,
  4682.08,
  5266.27,
  4877.21,
  None,
  4071.69,
  3541.27,
  3150.37,
  2888.95,
  2739.25,
  2597.27,
  1996.27,
  1837.88,
  1752.2,
  1535.68,
  1291.85,
  1020.25,
  959.86,
  990.44,
  613.82,
  723.7,
  5153.99,
  5492.71,
  4378.02,
  4725.72,
  4817.88,
  5072.03,
  4290.43,
  3591.64,
  3122.32,
  2832.4,
  2618.71,
  2658.03,
  2170.4,
  2049.3,
  1829.09,
  1276.44,
  1064.16,


  5703.18,
  5805.84,
  6161.79,
  6844.51,
  7100.59,
  7231.65,
  7259.58,
  6465.09,
  5885.0,
  6044.89,
  6281.44,
  6719.66,
  6829.65,
  6040.37,
  6097.06,
  5503.45,
  5401.74,
  5488.35,
  5533.89,
  5551.91,
  5344.62,
  5793.74,
  6039.23,
  6270.5,
  7151.36,
  6630.19,
  6724.75,
  7092.58,
  7055.69,
  6864.1,
  6134.46,
  6762.73,
  6800.53,
  6418.93,
  6539.49,
  6156.83,
  5947.13,
  5530.27,
  5431.85,
  5121.94,
  5313.41,
  5343.81,
  5519.14,
  6121.4,
  5712.73,
  5700.18,
  6048.27,
  6315.1,
  6765.95,
  6826.68,
  7770.77,
  5924.75,
  6813.74,
  6173.04,
  6373.56,
  6678.59,
  6345.09,
  6742.67,
  5918.04,
  5746.23,
  5382.99,
  5232.95,
  4968.73,
  5014.78,
  5183.24,
  5221.09,
  5258.13,
  5786.05,
  6079.63,
  6380.81,
  6576.33,
  6568.73,
  6519.92,
  6494.42,
  7767.08,
  6493.39,
  6320.59,
  6400.97,
  5823.87,
  5646.49,
  5629.78,
  5583.35,
  5281.58,
  5324.34,
  4755.36,
  4931.2,
  5036.13,
  5040.26,
  5179.16,
  5593.66,
  5694.33,
  619

  2430.82,
  2303.63,
  2350.03,
  2019.22,
  2214.94,
  2391.48,
  2556.75,
  2992.61,
  3473.0,
  3383.62,
  3665.45,
  3819.38,
  4007.15,
  4597.98,
  None,
  2009.77,
  1334.56,
  0.0]]


In [96]:
print(len(durations), len(durations[0]))

400 400


In [101]:
np.savetxt("durations.csv", np.array(durations), delimiter=",", fmt="%s")